# B0 map calculation tutorial

Start by importing the required libraries and defining some settings:

In [ ]:
# Ensure `data` and `methods` can be imported
import sys
sys.path.insert(0, '..')

# Other imports
import os
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt
import warnings

# UKRIN-MAPS modules
import data.fetch                  # Test data fetcher
from methods.B0Map import B0Map    # B0 mapping module

# Enable inline plotting; hide T2Star_Nottingham() RuntimeWarnings
%matplotlib inline
warnings.filterwarnings('ignore')

Tell python where to store the B0 map NIfTI file we will generate. We will store it in a directory called `b0_calculation_output` in the `tutorials` directory.

In [ ]:
# Initialise output path for T2star map
OUTPUT_DIR = os.path.join(os.getcwd(), "b0_calculation_output")
OUTPUT_PATH_1 = os.path.join(OUTPUT_DIR, 'B0MapSIEMENS-1.nii.gz')
OUTPUT_PATH_2 = os.path.join(OUTPUT_DIR, 'B0MapSIEMENS-2.nii.gz')

Retrieve B0 test data using our data fetcher.
This will return 4 variables:
* `magnitude`: a 4D array with the magnitude data at all echo times.
* `phase`: a 4D array with the phase data at all echo times.
* `affine`: the affine matrix for the data (needed to later write a new NIfTI file).
* `TEs`: an array containing the echo times, in seconds.

In [ ]:
# Fetch test data
magnitude_1, phase_1, affine_1, TEs_1 = data.fetch.b0_siemens_1()
magnitude_2, phase_2, affine_2, TEs_2 = data.fetch.b0_siemens_2()

Compute the B0 map using the `B0Map` class implemented in `methods`:

In [ ]:
# Compute T2star map
TEs_1 = TEs_1 * 1000  # convert TEs to ms
b0_1 = B0Map(phase_1, TEs_1)
TEs_2 = TEs_2 * 1000  # convert TEs to ms
b0_2 = B0Map(phase_2, TEs_2)

Display a central slice of the B0 map. Note the units are _rad/s_.

In [ ]:
# Display a central slice of the T2star map
b0_central_slice = np.rot90(b0_1[:,6,:], 1)
ax = plt.subplots()[1]
im = ax.imshow(b0_central_slice, vmin=-1000, vmax=1000)
plt.colorbar(im)
plt.show()

In [ ]:
# Display a central slice of the T2star map
b0_central_slice = np.rot90(b0_2[:,:,6], 1)
ax = plt.subplots()[1]
im = ax.imshow(b0_central_slice, vmin=-1000, vmax=1000)
plt.colorbar(im)
plt.show()

Save the resulting B0 map as a NIfTI file:

In [ ]:
# Save results as a new NifTI file
b0_nifti_object_1 = nib.Nifti1Image(b0_1, affine=affine_1)
nib.save(b0_nifti_object_1, OUTPUT_PATH_1)
b0_nifti_object_2 = nib.Nifti1Image(b0_2, affine=affine_2)
nib.save(b0_nifti_object_2, OUTPUT_PATH_2)